In [326]:
# 1. dataset <- titanic.csv, dan tampilkan
import pandas as pd 
import numpy as np 

dataset = pd.read_csv('titanic.csv')
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [327]:
# 2. data <- ambil test_dataset kolom fitur (Sex, Age, Pclass, Fare).
data = dataset[['Sex', 'Age', 'Pclass', 'Fare']]
data.head()

,Sex,Age,Pclass,Fare
0,male,22.0,3,7.2500
1,female,38.0,1,71.2833
2,female,26.0,3,7.9250
3,female,35.0,1,53.1000
4,male,35.0,3,8.0500


In [328]:
#3. train_data <- ambil fitur (Sex, Pclass, Fare, Survived) pada data yang Age != null
train_data = dataset[dataset['Age'].notnull()]
train_data = train_data[['Sex', 'Pclass', 'Fare', 'Survived']]
train_data.head()

,Sex,Pclass,Fare,Survived
0,male,3,7.2500,0
1,female,1,71.2833,1
2,female,3,7.9250,1
3,female,1,53.1000,1
4,male,3,8.0500,0


In [329]:
#4. train_label <- ambil fitur (Age) pada data yang Age <- null
train_label = dataset[dataset['Age'].notnull()]['Age']
train_label.head()

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
Name: Age, dtype: float64

In [330]:
#5. test_data <- ambil fitur (Sex, Pclass, Fare, Survived) pada data yang Age=null
test_data = dataset[dataset['Age'].isnull()]
test_data = test_data[['Sex', 'Pclass', 'Fare', 'Survived']]
test_data.head()

,Sex,Pclass,Fare,Survived
5,male,3,8.4583,0
17,male,2,13.0000,1
19,female,3,7.2250,1
26,male,3,7.2250,0
28,female,3,7.8792,1


In [331]:
# Mengonversi kolom 'Sex' menjadi numerik
train_data.loc[:, 'Sex'] = train_data['Sex'].map({'male': 1, 'female': 0})
test_data.loc[:, 'Sex'] = test_data['Sex'].map({'male': 1, 'female': 0})

In [332]:
# 6. train_data <- lakukan normalisasi pada train_data dengan Min-Max 0-1 (catat nilai min dan max setiap atribut)
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()

train_data = train_data.copy()
train_data[['Sex', 'Pclass', 'Fare', 'Survived']] = sc.fit_transform(train_data[['Sex', 'Pclass', 'Fare', 'Survived']])

# Get the min and max values for each attribute
min_values = train_data[['Sex', 'Pclass', 'Fare', 'Survived']].min()
max_values = train_data[['Sex', 'Pclass', 'Fare', 'Survived']].max()

# Display the normalized train_data and the min/max values
train_data.head(), min_values, max_values

(   Sex  Pclass      Fare  Survived
 0  1.0     1.0  0.014151       0.0
 1  0.0     0.0  0.139136       1.0
 2  0.0     1.0  0.015469       1.0
 3  0.0     0.0  0.103644       1.0
 4  1.0     1.0  0.015713       0.0,
 Sex         0.0
 Pclass      0.0
 Fare        0.0
 Survived    0.0
 dtype: float64,
 Sex         1.0
 Pclass      1.0
 Fare        1.0
 Survived    1.0
 dtype: float64)

In [333]:
# 7. test_data <- lakukan normalisasi pada test_data dengan Min-Max 0-1 (dengan nilai min dan max setiap atribut pada Langkah 6)
# Extract the features: 'Sex', 'Pclass', 'Fare', 'Survived'
test_data = test_data[['Sex', 'Pclass', 'Fare', 'Survived']]
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1}).fillna(1) 

test_data = test_data.copy()
test_data['Sex'] = (test_data['Sex'] - min_values['Sex']) / (max_values['Sex'] - min_values['Sex'])
test_data['Pclass'] = (test_data['Pclass'] - min_values['Pclass']) / (max_values['Pclass'] - min_values['Pclass'])
test_data['Fare'] = (test_data['Fare'] - min_values['Fare']) / (max_values['Fare'] - min_values['Fare'])
test_data['Survived'] = (test_data['Survived'] - min_values['Survived']) / (max_values['Survived'] - min_values['Survived'])

# Display the normalized test_data
test_data.head()

,Sex,Pclass,Fare,Survived
5,1.0,3.0,8.4583,0.0
17,1.0,2.0,13.0000,1.0
19,1.0,3.0,7.2250,1.0
26,1.0,3.0,7.2250,0.0
28,1.0,3.0,7.8792,1.0


In [334]:
# 8. train_data dengan 3-NN (output memprediksi nilai Age)
from sklearn.neighbors import KNeighborsRegressor

knn_regressor = KNeighborsRegressor(n_neighbors=3)
knn_regressor.fit(train_data, train_label)
class_result = knn_regressor.predict(test_data)

In [335]:
# 9. data (Age) <- lakukan pengisian missing values pada data yang Age=null dengan nilai class_result
data.loc[data['Age'].isnull(), 'Age'] = class_result

In [336]:
# 10. test_dataset <- titanic_test.csv
test_dataset = pd.read_csv('titanic_test.csv')
test_dataset.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [337]:
# 11. train_data <- ambil fitur (Sex, Age, Pclass, Fare) dari data
train_data = data[['Sex', 'Age', 'Pclass', 'Fare']]
train_data.head()

,Sex,Age,Pclass,Fare
0,male,22.0,3,7.2500
1,female,38.0,1,71.2833
2,female,26.0,3,7.9250
3,female,35.0,1,53.1000
4,male,35.0,3,8.0500


In [338]:
# 12. train_label <- ambil fitur (Survived) dari data
train_label = dataset['Survived']
train_label.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [339]:
# 13. test_data <- ambil test_dataset kolom fitur (Sex, Age, Pclass, Fare). Hilangkan baris data yang terdapat missing values
test_data = test_dataset[['Sex', 'Age', 'Pclass', 'Fare']].dropna()

# Pastikan kolom 'Sex' pada test_data sudah diencode sama seperti pada train_data
test_data['Sex'] = test_data['Sex'].map({'male': 1, 'female': 0})
train_data['Sex'] = test_data['Sex'].map({'male': 1, 'female': 0})

In [340]:
# 14. test_label <- titanic_testlabel.csv (urutan sesuai test_data no.13)
test_label = pd.read_csv('titanic_testlabel.csv')
test_label = test_label.loc[test_data.index]
test_label.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [341]:
# 15. train_data <- lakukan normalisasi pada train_data dengan Min-Max 0-1 (catat nilai min dan max setiap atribut)
train_data_normalized = sc.fit_transform(train_data)
print(train_data_normalized)

[[       nan 0.27117366 1.         0.01415106]
 [       nan 0.4722292  0.         0.13913574]
 [       nan 0.32143755 1.         0.01546857]
 ...
 [       nan 0.43871995 1.         0.04577135]
 [       nan 0.32143755 0.         0.0585561 ]
 [       nan 0.39683338 1.         0.01512699]]


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [342]:
# 16. test_data <- lakukan normalisasi pada test_data dengan Min-Max 0-1 (dengan nilai min dan max setiap atribut pada Langkah 15)
test_data_normalized = sc.transform(test_data)
print(test_data_normalized)

[[       nan 0.4282483  1.         0.01528158]
 [       nan 0.58532295 1.         0.01366309]
 [       nan 0.77381252 0.5        0.01890874]
 ...
 [       nan 0.34656949 1.         0.01517579]
 [       nan 0.48479517 0.         0.21255864]
 [       nan 0.47851219 1.         0.01415106]]


In [343]:
# 17. class_result <- Lakukan klasifikasi test_data terhadap train_data dengan 3-NN (output memakai class pada train_label)
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_data_normalized, train_label)
class_result = knn.predict(test_data_normalized)

ValueError: Input X contains NaN.
KNeighborsClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [305]:
# 18. error  Bandingkan hasil klasifikasi class_result dengan test_label. 
# Pastikan test_label adalah array satu dimensi
test_label = test_label['Survived']

print(test_label)

error = np.sum(class_result != test_label.values.ravel())  # Hitung jumlah error, yaitu data yang berbeda antara class_result dan test_label

# Output hasil error
print(f'Jumlah error: {error}')


KeyError: 'Survived'

In [306]:

# 19. error_ratio  error dibagi jumlah test_data, dikali 100 (%)
error_ratio = (error / len(test_data)) * 100

# Cetak hasil akhir
print(f'Jumlah error: {error}')
print(f'Error ratio: {error_ratio:.2f}%')

NameError: name 'error' is not defined